In [ ]:
import fitz  # PyMuPDF
import re
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter # initial_text_split 에서 사용
import os
import json
import uuid # llm_based_semantic_chunking_for_dev_reqs 에서 사용

# --- OpenAI API 클라이언트 초기화 (사용자 기존 코드) ---
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")
client = OpenAI(api_key=api_key)
LLM_MODEL_FOR_PARSING = "gpt-4o" # 목차 파싱 및 요구사항 추출에 사용할 모델

# --- 사용자 기존 함수 정의 부분 (여기에 extract_text_with_page_info, initial_text_split, llm_based_semantic_chunking_for_dev_reqs 가 있어야 함) ---

def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하고, 각 페이지 시작에 페이지 번호를 표기합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        # 페이지 마커를 맨 앞에 추가하고, 실제 텍스트 내용과 명확히 구분되도록 줄바꿈 추가
        full_text_with_pages.append(f"---PAGE_START_{page_num+1}---\n{text.strip()}")
    return "\n".join(full_text_with_pages), document.page_count

def initial_text_split(text, chunk_size=4000, chunk_overlap=200):
    """
    텍스트를 LLM이 처리할 수 있는 크기로 초기 분할합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n---PAGE_START_\d+---\n", "\n\n", "\n", ". ", "? ", "! ", " ", ""],
        keep_separator=True # 페이지 마커 유지를 위해 True 권장
    )
    docs = text_splitter.create_documents([text])
    return docs

# llm_based_semantic_chunking_for_dev_reqs 함수 정의 (사용자 스크립트 내용)
# def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model="gpt-4o"): ...
# (이전에 제공된 프롬프트와 로직을 그대로 사용한다고 가정)
def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    final_semantic_chunks = []
    req_id_counter = 0

    system_prompt = """
    당신은 RFP(제안요청서) 문서에서 시스템 요구사항을 전문적으로 분석하고 개발 표준에 맞춰 구조화하는 시니어 비즈니스 분석가입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 특정 섹션이며, 여기서 모든 기능적, 비기능적, 성능, 보안, 데이터, 제약사항 요구사항을 추출해야 합니다.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.

    -   **id**: 고유 식별자. 예를 들어, 기능적 요구사항은 "FUNC-001", 비기능적은 "NFR-001", 보안은 "SEC-001"과 같이 유형 접두사와 일련번호를 결합하여 생성하십시오. (RFP 원문에 ID가 있다면 그것을 우선 사용)
    -   **type**: 요구사항의 유형. 다음 중 하나를 선택: "기능적", "비기능적", "성능", "보안", "데이터", "제약사항", "시스템 장비 구성", "컨설팅", "테스트", "품질", "프로젝트 관리", "프로젝트 지원", "기타". 텍스트에 명시된 내용에 따라 가장 적합한 유형을 선택하십시오. (예: "시스템 장비 구성요구사항", "컨설팅 요구사항" 등 PDF의 분류명 활용)
    -   **description**: 요구사항에 대한 명확하고 간결한 설명. 원본 PDF의 '요구사항 명칭'과 '정의', '상세설명/세부내용'을 종합하여 개발 친화적인 형태로 작성하십시오. 하나의 요구사항은 하나의 독립적인 기능 또는 특성을 나타내야 합니다.
    -   **acceptance_criteria**: 이 요구사항이 충족되었음을 검증할 수 있는 구체적인 테스트 조건이나 결과. 1~2개의 명확한 문장으로 서술하십시오. 원본 텍스트에 직접적인 검증 기준이 없더라도, 요구사항의 내용에 기반하여 합리적으로 추론하여 작성하십시오. (예: "사용자가 [기능]을 수행했을 때, [예상 결과]가 나타난다.")
    -   **priority**: 요구사항의 중요도. 다음 중 하나를 선택: "필수", "높음", "중간", "낮음". (텍스트에 명시되어 있다면 그대로 사용, 아니면 일반적으로 "필수"로 추정)
    -   **responsible_module**: 이 요구사항이 주로 영향을 미치거나 구현될 것으로 예상되는 시스템/애플리케이션의 주요 모듈 또는 영역 (예: "로그인 모듈", "결제 시스템", "관리자 페이지", "데이터베이스"). 텍스트 내용을 기반으로 추론하십시오.
    -   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트. 페이지 구분자(`---PAGE_START_N---`)를 참조하여 정확한 페이지 번호를 파싱하십시오.
    -   **raw_text_snippet**: 이 요구사항이 포함된 원본 텍스트 스니펫. 해당 요구사항을 추출하는 데 사용된 원본 문장 또는 단락(예: 표의 해당 행 전체 내용)을 포함하십시오.

    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 요구사항이 없다면 빈 배열 `[]`을 반환하십시오.
    불필요한 서론, 사업 배경, 계약 조건, 제안 지침 등은 요구사항으로 추출하지 마십시오. PDF의 '상세 요구사항' 목록에 있는 구조화된 항목들 위주로 추출해주십시오.
    """

    for i, chunk_doc in enumerate(initial_chunks):
        chunk_text = chunk_doc.page_content
        print(f"--- LLM 처리 중 (요구사항 상세 추출): 청크 {i+1}/{len(initial_chunks)} (길이: {len(chunk_text)}자) ---")

        page_numbers_in_chunk = sorted(list(set(
            int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text)
        )))

        user_prompt = f"""
        다음은 RFP 문서의 일부입니다. 이 부분에서 모든 시스템 요구사항을 개발자 표준에 맞춰 JSON 형식으로 추출해 주세요.
        주어진 텍스트 내의 "요구사항 고유번호", "요구사항 명칭", "요구사항 분류", "정의", "상세설명/세부내용" 등의 명시적 필드를 최대한 활용하여 JSON 객체를 구성해주십시오.

        --- 텍스트 ---
        {chunk_text}
        """

        try:
            response = client_instance.chat.completions.create(
                model=llm_model,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0
            )

            llm_response_content = response.choices[0].message.content
            try:
                extracted_data_outer = json.loads(llm_response_content)
                extracted_data_list = []
                if isinstance(extracted_data_outer, list):
                    extracted_data_list = extracted_data_outer
                elif isinstance(extracted_data_outer, dict):
                    # 딕셔너리 값 중에 리스트를 찾아 첫 번째 것을 사용 (일반적인 LLM 응답 패턴)
                    for key_in_dict in extracted_data_outer:
                        if isinstance(extracted_data_outer[key_in_dict], list):
                            extracted_data_list = extracted_data_outer[key_in_dict]
                            break
                    if not extracted_data_list: # 그래도 못찾으면 경고
                        print(f"경고: LLM이 JSON 객체를 반환했으나, 그 안에 예상된 요구사항 리스트를 찾지 못했습니다. 객체 키: {list(extracted_data_outer.keys())}")
                else:
                    print(f"경고: LLM으로부터 예상치 못한 JSON 형식 응답 (리스트 또는 객체 내 리스트가 아님). {llm_response_content[:100]}...")
                    continue
            except json.JSONDecodeError as e_inner:
                print(f"LLM 응답 내용 JSON 파싱 오류 (내부 시도): {e_inner}. 응답: {llm_response_content[:500]}...")
                continue

            for req_idx, req in enumerate(extracted_data_list):
                # 'id'가 없거나 비어있으면 생성, 또는 원본 ID 우선 사용
                original_id = req.get('id') # LLM이 원본 ID (ECR-001 등)를 id 필드에 넣어줬길 기대
                if not original_id:
                    req_type_prefix = "REQ"
                    req_type = req.get('type', '기타').strip()
                    if "기능" in req_type: req_type_prefix = "FUNC"
                    elif "비기능" in req_type: req_type_prefix = "NFR"
                    elif "성능" in req_type: req_type_prefix = "PERF"
                    elif "보안" in req_type: req_type_prefix = "SEC"
                    elif "데이터" in req_type: req_type_prefix = "DATA"
                    elif "제약" in req_type: req_type_prefix = "CONST"
                    elif "장비" in req_type: req_type_prefix = "ECR"
                    elif "컨설팅" in req_type: req_type_prefix = "CSR"
                    elif "테스트" in req_type: req_type_prefix = "TER"
                    elif "품질" in req_type: req_type_prefix = "QUR"
                    elif "관리" in req_type: req_type_prefix = "PMR"
                    elif "지원" in req_type: req_type_prefix = "PSR"
                    req_id_counter += 1
                    req['id'] = f"{req_type_prefix}-{req_id_counter:03d}"

                if 'source_pages' not in req or not req['source_pages']:
                    req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else []

                if 'raw_text_snippet' not in req or not req['raw_text_snippet']:
                    req['raw_text_snippet'] = f"청크 {i+1}에서 추출됨. 원본 청크 일부: {chunk_text[:200]}..." # 개선 필요

                req['type'] = req.get('type', '기타')
                req['description'] = req.get('description', '설명 없음')
                req['acceptance_criteria'] = req.get('acceptance_criteria', '해당하는 경우 명시')
                req['priority'] = req.get('priority', '필수')
                req['responsible_module'] = req.get('responsible_module', '미정')
                final_semantic_chunks.append(req)

        except json.JSONDecodeError as e:
            print(f"LLM 응답 JSON 파싱 오류: {e}. 응답: {llm_response_content[:500]}...")
        except Exception as e:
            print(f"LLM API 호출 또는 처리 중 오류 발생: {e}")

    return final_semantic_chunks


# --- LLM으로 목차 파싱하는 함수 (이전 답변에서 제공 및 개선된 버전) ---
def parse_toc_with_llm(toc_raw_text, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    """
    LLM을 사용하여 목차 원문 텍스트를 파싱하여 구조화된 목차 항목을 추출합니다.
    """
    system_prompt = """
    당신은 PDF 문서에서 추출된 목차(Table of Contents)의 원시 텍스트를 분석하는 전문가입니다.
    주어진 텍스트를 파싱하여 각 목차 항목의 제목, 페이지 번호, 그리고 해당 항목이 '요구사항' 관련 내용을 담고 있을 가능성을 분석하여 JSON 형태로 반환해야 합니다.
    JSON의 최상위 레벨은 "toc_entries"라는 키를 가진 객체여야 하고, 그 키의 값은 목차 항목 객체들의 리스트여야 합니다.
    """
    user_prompt = f"""
    다음은 PDF에서 추출한 목차로 추정되는 텍스트입니다:

    --- 목차 원문 텍스트 시작 ---
    {toc_raw_text}
    --- 목차 원문 텍스트 끝 ---

    이 텍스트를 분석하여 JSON 객체를 반환해주세요. 이 객체는 "toc_entries"라는 키를 가져야 하며,
    이 키의 값은 각 목차 항목을 나타내는 객체들의 리스트여야 합니다.
    각 목차 항목 객체는 다음 키를 가져야 합니다:
    - "title": (문자열) 목차 항목의 전체 제목. 제목 앞의 번호(예: "1.", "II.", "가.")도 포함해주세요.
    - "page": (정수) 해당 항목의 시작 페이지 번호.
    - "is_requirement_related": (불리언) 제목이나 내용을 볼 때, 해당 항목이 '요구사항', '과업 범위', '제안 요청 상세', '기능 명세', '기술 요건' 등과 관련된 내용을 다룰 가능성이 높으면 true, 그렇지 않으면 false로 설정해주세요.

    예시 JSON 출력 형식:
    {{
      "toc_entries": [
        {{
          "title": "1. 사업 개요",
          "page": 5,
          "is_requirement_related": false
        }},
        {{
          "title": "III. 제안요청 내용",
          "page": 6,
          "is_requirement_related": true
        }},
        {{
          "title": "3. 상세 요구사항",
          "page": 11,
          "is_requirement_related": true
        }},
        {{
          "title": "* 보안 요구사항 별표",
          "page": 63,
          "is_requirement_related": true
        }}
      ]
    }}

    만약 주어진 텍스트가 유효한 목차로 보이지 않거나 항목을 전혀 파싱할 수 없다면,
    "toc_entries" 키의 값으로 빈 리스트 `[]`를 포함하는 JSON 객체를 반환해주세요. (예: {{"toc_entries": []}})
    """
    llm_response_content = "" # 오류 발생 시 로깅을 위해 미리 선언
    try:
        response = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        llm_response_content = response.choices[0].message.content
        # print(f"DEBUG: LLM RAW RESPONSE FOR TOC: {llm_response_content}")

        parsed_data = json.loads(llm_response_content)

        extracted_list = []
        if isinstance(parsed_data, dict) and "toc_entries" in parsed_data and isinstance(parsed_data["toc_entries"], list):
            extracted_list = parsed_data["toc_entries"]
        else:
            print(f"LLM 응답이 예상된 'toc_entries' 리스트를 포함하는 객체 형식이 아닙니다. 응답: {llm_response_content[:200]}")
            return [] # 빈 리스트 반환

        valid_entries = []
        for entry in extracted_list:
            if isinstance(entry, dict) and 'title' in entry and 'page' in entry:
                try:
                    entry['page'] = int(entry['page'])
                    entry['is_requirement_related'] = bool(entry.get('is_requirement_related', False)) # 명확히 불리언으로
                    valid_entries.append(entry)
                except ValueError:
                    print(f"경고: 페이지 번호 '{entry.get('page')}'를 정수로 변환할 수 없습니다. 항목 건너뜀: {entry.get('title')}")
                    continue
            else:
                print(f"경고: 필수 키(title, page)가 누락된 항목입니다. 건너뜀: {entry}")

        return valid_entries

    except json.JSONDecodeError as e:
        print(f"LLM 목차 파싱 응답 JSON 파싱 오류: {e}. 응답 미리보기: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"LLM API 호출 또는 처리 중 오류 발생 (목차 파싱): {e}")
        return []

# --- 새로운 헬퍼 함수들 ---
def extract_text_for_pages(full_text_with_pages, start_page_num, end_page_num):
    """
    전체 텍스트에서 특정 페이지 범위의 텍스트만 추출합니다.
    페이지 마커 '---PAGE_START_N---'를 사용합니다.
    """

    # 페이지 범위 유효성 검사 (end_page_num이 start_page_num보다 작을 수 없음)
    if end_page_num < start_page_num:
        # print(f"경고: 끝 페이지({end_page_num})가 시작 페이지({start_page_num})보다 작습니다. 빈 텍스트를 반환합니다.")
        return ""

    # 패턴 구성: 시작 페이지부터 (끝 페이지 + 1) 직전까지 모든 내용을 포함
    # re.escape를 사용하여 페이지 마커의 특수 문자를 이스케이프할 필요는 여기서는 없음
    # DOTALL 플래그를 사용하여 \n도 .에 매치되도록 함

    # 시작점 찾기
    start_marker = f"---PAGE_START_{start_page_num}---"
    start_match = re.search(re.escape(start_marker), full_text_with_pages) # 마커 자체를 찾아야 함

    if not start_match:
        # print(f"경고: 시작 마커 '{start_marker}'를 찾을 수 없습니다.")
        return ""

    text_from_start_page = full_text_with_pages[start_match.start():]

    # 끝점 찾기 (다음 페이지 마커 ---PAGE_START_{end_page_num + 1}---)
    # end_page_num이 문서의 마지막 페이지일 수도 있으므로, end_page_num + 1 마커가 없을 수 있음
    end_marker_exclusive = f"---PAGE_START_{end_page_num + 1}---"
    end_match = re.search(re.escape(end_marker_exclusive), text_from_start_page)

    if end_match:
        return text_from_start_page[:end_match.start()]
    else:
        # end_page_num + 1 마커를 찾지 못하면, start_page_num부터 문서 끝까지 반환 (또는 start_page_num ~ end_page_num의 마지막 내용까지)
        # 이 경우는 end_page_num이 마지막 페이지이거나, 그 이후 페이지 마커가 없는 경우.
        # 좀 더 정확하게 하려면, end_page_num까지의 모든 내용을 가져와야 함.
        # 그러나 페이지 마커 구조상, end_page_num의 내용은 end_page_num+1 마커 전까지임.
        # 따라서 end_match가 없으면 text_from_start_page 전체가 해당 범위임.
        return text_from_start_page


def get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages):
    """
    LLM으로 파싱된 목차에서 '상세 요구사항' 및 '보안 요구사항' 관련 섹션 정보를 추출합니다.
    """
    target_sections = []

    # 페이지 번호 기준으로 목차 정렬 (LLM이 순서대로 안 줄 수도 있으므로)
    sorted_toc = sorted(parsed_toc_entries, key=lambda x: x.get('page', 0))

    # 주요 키워드 - 사용자가 제공한 PDF 목차 기반
    # "III. 제안요청 내용" 안에 "3. 상세 요구사항"이 있으므로, "상세 요구사항"을 찾는 것이 더 정확함.
    keywords_to_find = {
        "상세 요구사항": {"min_pages": 5}, # 최소한 이정도는 될 것이라는 기대
        "보안 요구사항 별표": {"min_pages": 1} # 별표는 짧을 수도 있음
        # 필요시 다른 주요 섹션 키워드 추가 가능
    }

    for i, entry in enumerate(sorted_toc):
        entry_title = entry.get('title', '').strip()
        entry_page = entry.get('page', 0)

        for keyword, props in keywords_to_find.items():
            if keyword in entry_title:
                start_page = entry_page
                end_page = total_pages # 기본값: 문서 끝까지

                # 다음 목차 항목의 시작 페이지 - 1을 현재 섹션의 끝 페이지로 설정
                # 단, 다음 항목이 현재 항목과 같은 페이지에서 시작하면 안됨 (하위 항목일 수 있으므로)
                # 좀 더 정교한 로직: 다음 '주요' 항목을 찾아야 함.
                # 여기서는 일단 다음 항목의 시작 페이지를 사용.
                # 만약 다음 항목이 현재 항목의 하위 항목처럼 보이면 (예: "3. 상세 요구사항" 다음 "3.1 기능 요구사항")
                # 그 하위 항목의 범위를 포함하도록 확장하거나, 아니면 정말 다음 *다른* 주요 섹션까지 봐야 함.
                # 현재 LLM 프롬프트는 is_requirement_related로 판단하므로, 이를 우선적으로 신뢰.

                # 다음 'is_requirement_related=False'인 섹션 또는 다음 주요 섹션(로마숫자/대문자 알파벳 등)을 찾아 end_page 설정
                # 또는 단순히 다음 목차 항목의 시작 페이지 - 1 로 설정
                next_major_section_page = total_pages + 1 # 충분히 큰 값
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page : # 현재 섹션 이후의 페이지만 고려
                        # 여기서 '주요 섹션'을 판단하는 기준이 필요함 (예: 로마 숫자, 대문자 번호 등)
                        # 또는 is_requirement_related=False 인 첫번째 섹션
                        # 또는 단순히 다음 목차 항목
                        next_major_section_page = next_entry_page
                        break

                end_page = min(total_pages, next_major_section_page -1)

                # 시작 페이지와 끝 페이지 유효성 확보
                if end_page < start_page:
                    end_page = start_page

                # 페이지 수가 너무 적으면 해당 섹션 전체를 포함 (예: 11페이지로 나왔는데 실제로는 62까지일 수 있음)
                # 이 부분은 heuristic이므로 주의. 더 좋은 방법은 LLM에게 섹션의 끝을 명확히 묻는 것.
                # 여기서는 min_pages 이상은 되어야 유의미하다고 가정.
                # if (end_page - start_page + 1) < props.get("min_pages", 1) and (start_page + props.get("min_pages", 1) -1) <= total_pages :
                #     pass # end_page = start_page + props.get("min_pages", 1) -1 # 최소 페이지 강제는 위험할 수 있음

                target_sections.append({
                    'title': entry_title,
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 파싱 목차 기반: '{entry_title}' 섹션 (페이지 {start_page}-{end_page}) 식별")
                # 찾은 키워드는 중복 추가 방지 (더 구체적인 항목이 먼저 찾아지도록 keywords_to_find 순서 중요)
                break

    # 만약 아무것도 못찾았지만, is_requirement_related=True 인 항목들이 있다면 그것들을 사용
    if not target_sections:
        print("키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...")
        for i, entry in enumerate(sorted_toc):
            if entry.get('is_requirement_related'):
                start_page = entry.get('page',0)
                end_page = total_pages
                # 위와 동일한 로직으로 end_page 계산
                next_major_section_page = total_pages + 1
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page:
                        if not next_entry.get('is_requirement_related', False): # 다음 주요 섹션이 요구사항 관련이 아니면
                            next_major_section_page = next_entry_page
                            break
                        # 또는 다음 항목이 현재 항목보다 상위 레벨이면 (예: "3.1" 다음 "4.")
                        # 이 부분은 제목의 번호 체계를 분석해야 해서 복잡함. LLM의 is_requirement_related를 신뢰.

                end_page = min(total_pages, next_major_section_page - 1)
                if end_page < start_page: end_page = start_page

                target_sections.append({
                    'title': entry.get('title', '요구사항 관련 섹션'),
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 'is_requirement_related' 플래그 기반: '{entry.get('title')}' 섹션 (페이지 {start_page}-{end_page}) 식별")

    # 중복 제거 및 병합 (예: "III. 제안요청 내용"과 그 하위 "3. 상세 요구사항"이 모두 선택된 경우)
    # 여기서는 단순화를 위해 중복된 페이지 범위가 있다면 가장 포괄적인 것을 선택하거나,
    # 또는 가장 구체적인 "상세 요구사항"을 우선. 지금은 일단 나온대로 반환.
    # 더 나은 방법은, '상세 요구사항'이 나왔으면 그게 더 우선순위가 높다고 처리.

    if not target_sections:
        print("경고: LLM 파싱 목차에서 주요 요구사항 섹션을 찾지 못했습니다. 전체 문서를 대상으로 할 수 있습니다.")
        return [{'title': '전체 문서 (목차 분석 실패)', 'start_page': 1, 'end_page': total_pages}]

    return target_sections


def get_toc_raw_text_from_full_text(full_text_with_pages, toc_page_numbers=[2,3]):
    """
    전체 텍스트에서 지정된 목차 페이지들의 텍스트만 추출합니다.
    """
    toc_texts = []
    for page_num in toc_page_numbers:
        # 페이지 마커를 찾아서 해당 페이지의 텍스트를 추출
        # (end_page_num + 1) 마커를 찾아서 그 전까지의 내용을 가져옴
        page_content = extract_text_for_pages(full_text_with_pages, page_num, page_num)
        if page_content:
            # 페이지 마커 제거 (이미 extract_text_for_pages가 마커 다음부터 가져오지만, 혹시 몰라서)
            # 실제로는 페이지 마커 이후의 텍스트만 필요.
            # `extract_text_for_pages`는 마커를 포함해서 반환할 수 있으므로, 마커 이후 내용만 사용.
            marker = f"---PAGE_START_{page_num}---\n"
            if page_content.startswith(marker):
                toc_texts.append(page_content[len(marker):])
            else: # 마커가 없는 경우 (예상치 않음)
                toc_texts.append(page_content)
        else:
            print(f"경고: 목차 페이지로 지정된 {page_num} 페이지에서 텍스트를 찾을 수 없습니다.")

    if not toc_texts:
        return None
    return "\n".join(toc_texts)

# --- 메인 실행 블록 ---
if __name__ == "__main__":
    pdf_file_path = "./docs/제주은행 모바일뱅킹 재구축사업.pdf" # 실제 파일 경로
    if not os.path.exists(pdf_file_path):
        print(f"오류: PDF 파일을 찾을 수 없습니다 - {pdf_file_path}")
        exit()

    print("1. PDF 전체 텍스트 추출 중...")
    full_document_text, total_pages = extract_text_with_page_info(pdf_file_path)
    if not full_document_text or total_pages == 0:
        print(f"오류: PDF에서 텍스트를 추출하지 못했거나 페이지 수가 0입니다 ({pdf_file_path}).")
        exit()
    print(f"   PDF 전체 텍스트 추출 완료. 총 {total_pages} 페이지, 전체 텍스트 길이: {len(full_document_text)}자.")

    print("\n2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...")
    # 사용자 PDF 분석 결과 PAGE 2, 3에 목차가 있음
    toc_raw_text = get_toc_raw_text_from_full_text(full_document_text, toc_page_numbers=[2, 3])

    target_sections_for_extraction = []
    if toc_raw_text:
        print(f"   목차 원문 텍스트 추출 완료 (길이: {len(toc_raw_text)}자).")
        print("\n3. LLM을 사용하여 목차(ToC) 파싱 중...")
        parsed_toc_entries = parse_toc_with_llm(toc_raw_text, client, llm_model=LLM_MODEL_FOR_PARSING)

        if parsed_toc_entries:
            print(f"   LLM 목차 파싱 완료. {len(parsed_toc_entries)}개의 목차 항목 식별.")
            print("\n4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...")
            target_sections_for_extraction = get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages)
        else:
            print("   LLM 목차 파싱 실패. 전체 문서를 대상으로 분석합니다.")
            target_sections_for_extraction = [{'title': '전체 문서 (LLM 목차 파싱 실패)', 'start_page': 1, 'end_page': total_pages}]
    else:
        print("   목차 원문 텍스트 추출 실패. 전체 문서를 대상으로 분석합니다.")
        target_sections_for_extraction = [{'title': '전체 문서 (목차 원문 추출 실패)', 'start_page': 1, 'end_page': total_pages}]

    if not target_sections_for_extraction: # 만약을 위해 한번 더 체크
        print("오류: 분석할 대상 섹션을 정의하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    print("\n5. 식별된 주요 섹션으로부터 텍스트 결합 중...")
    all_requirements_related_text_parts = []
    for section_info in target_sections_for_extraction:
        print(f"   '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']}) 텍스트 추출 시도...")
        section_text = extract_text_for_pages(full_document_text, section_info['start_page'], section_info['end_page'])
        if section_text:
            all_requirements_related_text_parts.append(section_text)
            print(f"       '{section_info['title']}' 섹션 텍스트 추출 완료 (길이: {len(section_text)}자).")
        else:
            print(f"       경고: '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']})에서 텍스트를 추출하지 못했습니다.")

    if not all_requirements_related_text_parts:
        print("오류: 주요 요구사항 섹션에서 텍스트를 전혀 추출하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    combined_requirements_text = "\n\n".join(all_requirements_related_text_parts) # 섹션 간 구분을 위해 두 줄바꿈
    print(f"   주요 섹션 텍스트 결합 완료. 총 길이: {len(combined_requirements_text)}자.")

    print("\n6. 결합된 텍스트 초기 분할 중...")
    # chunk_size는 LLM의 context window와 한글 토큰 소모를 고려하여 설정
    initial_chunks = initial_text_split(combined_requirements_text, chunk_size=3500, chunk_overlap=350)
    print(f"   총 {len(initial_chunks)}개의 초기 청크가 생성되었습니다.")

    if not initial_chunks:
        print("오류: 텍스트 분할 결과 청크가 없습니다. 프로그램을 종료합니다.")
        exit()

    print("\n7. LLM을 이용한 개발자 표준 요구사항 상세 추출 시작...")
    extracted_requirements = llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model=LLM_MODEL_FOR_PARSING)

    print("\n8. 후처리: ID 중복 제거 및 최종 정렬 (옵션)...")
    # (기존 스크립트의 후처리 로직 사용 또는 필요시 개선)
    unique_requirements_by_id = {}
    processed_requirements = []
    if extracted_requirements:
        for req in extracted_requirements:
            req_id = req.get('id', str(uuid.uuid4())) # ID가 없다면 UUID로 임시 ID
            if req_id not in unique_requirements_by_id:
                unique_requirements_by_id[req_id] = req
            else:
                # ID가 중복될 경우, 설명을 합치거나 페이지 정보를 합치는 등의 로직 추가 가능
                # 여기서는 간단히 첫 번째 것만 유지하거나, 필요에 따라 업데이트
                # 예: 페이지 정보 병합
                existing_req = unique_requirements_by_id[req_id]
                new_pages = req.get('source_pages', [])
                if new_pages:
                    existing_req_pages = existing_req.get('source_pages', [])
                    existing_req['source_pages'] = sorted(list(set(existing_req_pages + new_pages)))
                print(f"중복 ID '{req_id}' 감지. 정보 업데이트 시도.")
        processed_requirements = list(unique_requirements_by_id.values())
    else:
        print("   추출된 요구사항이 없습니다.")

    # 최종 결과 출력 또는 저장
    print(f"\n--- 최종 추출된 고유 요구사항 수: {len(processed_requirements)}개 ---")
    if processed_requirements:
        print("\n--- 추출된 요구사항 미리보기 (상위 3개) ---")
        for i, req_item in enumerate(processed_requirements[:3]): # 'req' 변수명 충돌 피하기 위해 'req_item'으로 변경
            print(json.dumps(req_item, ensure_ascii=False, indent=2))
            print("-" * 30)

        output_filename = "extracted_requirements_final.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(processed_requirements, f, ensure_ascii=False, indent=4)
        print(f"\n추출된 요구사항이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("최종적으로 추출된 요구사항이 없습니다.")

1. PDF 전체 텍스트 추출 중...
   PDF 전체 텍스트 추출 완료. 총 54 페이지, 전체 텍스트 길이: 49066자.

2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...
   목차 원문 텍스트 추출 완료 (길이: 8670자).

3. LLM을 사용하여 목차(ToC) 파싱 중...
   LLM 목차 파싱 완료. 49개의 목차 항목 식별.

4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...
키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...
LLM 'is_requirement_related' 플래그 기반: '1. 제안 요청 개요' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.1. JBANK 시스템 재구축 추진 배경' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.2. JBANK 시스템 재구축 추진 목표' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.3. JBANK 시스템 재구축 사업 범위' 섹션 (페이지 2-4) 식별
LLM 'is_requirement_related' 플래그 기반: '4. 프로젝트 구축 범위 및 제안 요청 상세' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.1. JBANK 시스템 UI/UX 표준 체계 마련 : 1.3. JBANK 시스템 재구축 사업 범위 참조' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.2. JBANK 시스템 GRC 체계 마련(확인필요)' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.3. JBANK 시스템 개발 프레임워크 도입(확인필요)' 섹션 (페이지 9-11) 식별
LLM 'is

In [ ]:
import fitz  # PyMuPDF
import re
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter # initial_text_split 에서 사용
import os
import json
import uuid # llm_based_semantic_chunking_for_dev_reqs 에서 사용

# --- OpenAI API 클라이언트 초기화 (사용자 기존 코드) ---
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")
client = OpenAI(api_key=api_key)
LLM_MODEL_FOR_PARSING = "gpt-4o" # 목차 파싱 및 요구사항 추출에 사용할 모델

# --- 사용자 기존 함수 정의 부분 ---

def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하고, 각 페이지 시작에 페이지 번호를 표기합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        # 페이지 마커를 맨 앞에 추가하고, 실제 텍스트 내용과 명확히 구분되도록 줄바꿈 추가
        full_text_with_pages.append(f"---PAGE_START_{page_num+1}---\n{text.strip()}")
    return "\n".join(full_text_with_pages), document.page_count

def initial_text_split(text, chunk_size=4000, chunk_overlap=200):
    """
    텍스트를 LLM이 처리할 수 있는 크기로 초기 분할합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n---PAGE_START_\d+---\n", "\n\n", "\n", ". ", "? ", "! ", " ", ""],
        keep_separator=True # 페이지 마커 유지를 위해 True 권장
    )
    docs = text_splitter.create_documents([text])
    return docs

# llm_based_semantic_chunking_for_dev_reqs 함수 정의 (수정됨)
def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client_instance, llm_model="gpt-4o"):
    final_semantic_chunks = []
    req_id_counter = 0

    system_prompt = """
    당신은 RFP(제안요청서) 문서에서 시스템 요구사항을 전문적으로 분석하고 개발 표준에 맞춰 구조화하는 시니어 비즈니스 분석가입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 특정 섹션이며, 여기서 모든 기능적, 비기능적, 성능, 보안, 데이터, 제약사항 요구사항을 추출해야 합니다.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.

    -   **id**: 고유 식별자. 예를 들어, 기능적 요구사항은 "FUNC-001", 비기능적은 "NFR-001", 보안은 "SEC-001"과 같이 유형 접두사와 일련번호를 결합하여 생성하십시오. (RFP 원문에 ID가 있다면 그것을 우선 사용)
    -   **type**: 요구사항의 유형. 다음 중 하나를 선택: "기능적", "비기능적", "성능", "보안", "데이터", "제약사항", "시스템 장비 구성", "컨설팅", "테스트", "품질", "프로젝트 관리", "프로젝트 지원", "기타". 텍스트에 명시된 내용에 따라 가장 적합한 유형을 선택하십시오. (예: "시스템 장비 구성요구사항", "컨설팅 요구사항" 등 PDF의 분류명 활용)
    -   **description**: 요구사항에 대한 명확하고 간결한 설명. 원본 PDF의 '요구사항 명칭'과 '정의', '상세설명/세부내용'을 종합하여 개발 친화적인 형태로 작성하십시오. 하나의 요구사항은 하나의 독립적인 기능 또는 특성을 나타내야 합니다.
    -   **acceptance_criteria**: 이 요구사항이 충족되었음을 검증할 수 있는 구체적인 테스트 조건이나 결과. 1~2개의 명확한 문장으로 서술하십시오. 원본 텍스트에 직접적인 검증 기준이 없더라도, 요구사항의 내용에 기반하여 합리적으로 추론하여 작성하십시오. (예: "사용자가 [기능]을 수행했을 때, [예상 결과]가 나타난다.")
    -   **priority**: 요구사항의 중요도. 다음 중 하나를 선택: "필수", "높음", "중간", "낮음". (텍스트에 명시되어 있다면 그대로 사용, 아니면 일반적으로 "필수"로 추정)
    -   **responsible_module**: 이 요구사항이 주로 영향을 미치거나 구현될 것으로 예상되는 시스템/애플리케이션의 주요 모듈 또는 영역 (예: "로그인 모듈", "결제 시스템", "관리자 페이지", "데이터베이스"). 텍스트 내용을 기반으로 추론하십시오.
    -   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트. 페이지 구분자(`---PAGE_START_N---`)를 참조하여 정확한 페이지 번호를 파싱하십시오.
    -   **raw_text_snippet**: 이 요구사항이 포함된 원본 텍스트 스니펫. 해당 요구사항을 추출하는 데 사용된 원본 문장 또는 단락(예: 표의 해당 행 전체 내용)을 포함하십시오.

    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 요구사항이 없다면 빈 배열 `[]`을 반환하십시오.
    불필요한 서론, 사업 배경, 계약 조건, 제안 지침 등은 요구사항으로 추출하지 마십시오. PDF의 '상세 요구사항' 목록에 있는 구조화된 항목들 위주로 추출해주십시오.
    """

    for i, chunk_doc in enumerate(initial_chunks):
        chunk_text = chunk_doc.page_content
        print(f"--- LLM 처리 중 (요구사항 상세 추출): 청크 {i+1}/{len(initial_chunks)} (길이: {len(chunk_text)}자) ---")

        page_numbers_in_chunk = sorted(list(set(
            int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text)
        )))

        user_prompt = f"""
        다음은 RFP 문서의 일부입니다. 이 부분에서 모든 시스템 요구사항을 개발자 표준에 맞춰 JSON 형식으로 추출해 주세요.
        주어진 텍스트 내의 "요구사항 고유번호", "요구사항 명칭", "요구사항 분류", "정의", "상세설명/세부내용" 등의 명시적 필드를 최대한 활용하여 JSON 객체를 구성해주십시오.

        --- 텍스트 ---
        {chunk_text}
        """

        try:
            response = client_instance.chat.completions.create(
                model=llm_model,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0
            )

            llm_response_content = response.choices[0].message.content
            try:
                extracted_data_outer = json.loads(llm_response_content)
                extracted_data_list = []
                if isinstance(extracted_data_outer, list):
                    extracted_data_list = extracted_data_outer
                elif isinstance(extracted_data_outer, dict):
                    for key_in_dict in extracted_data_outer:
                        if isinstance(extracted_data_outer[key_in_dict], list):
                            extracted_data_list = extracted_data_outer[key_in_dict]
                            break
                    if not extracted_data_list:
                        print(f"경고: LLM이 JSON 객체를 반환했으나, 그 안에 예상된 요구사항 리스트를 찾지 못했습니다. 객체 키: {list(extracted_data_outer.keys())}")
                else:
                    print(f"경고: LLM으로부터 예상치 못한 JSON 형식 응답 (리스트 또는 객체 내 리스트가 아님). {llm_response_content[:100]}...")
                    continue
            except json.JSONDecodeError as e_inner:
                print(f"LLM 응답 내용 JSON 파싱 오류 (내부 시도): {e_inner}. 응답: {llm_response_content[:500]}...")
                continue

            for req_idx, req in enumerate(extracted_data_list):
                original_id = req.get('id')
                if not original_id:
                    req_type_prefix = "REQ"
                    req_type = req.get('type', '기타').strip()
                    if "기능" in req_type: req_type_prefix = "FUNC"
                    elif "비기능" in req_type: req_type_prefix = "NFR"
                    elif "성능" in req_type: req_type_prefix = "PERF"
                    elif "보안" in req_type: req_type_prefix = "SEC"
                    elif "데이터" in req_type: req_type_prefix = "DATA"
                    elif "제약" in req_type: req_type_prefix = "CONST"
                    elif "장비" in req_type: req_type_prefix = "ECR"
                    elif "컨설팅" in req_type: req_type_prefix = "CSR"
                    elif "테스트" in req_type: req_type_prefix = "TER"
                    elif "품질" in req_type: req_type_prefix = "QUR"
                    elif "관리" in req_type: req_type_prefix = "PMR"
                    elif "지원" in req_type: req_type_prefix = "PSR"
                    req_id_counter += 1
                    req['id'] = f"{req_type_prefix}-{req_id_counter:03d}"

                if 'source_pages' not in req or not req['source_pages']:
                    req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else []
                
                # --- MODIFICATION START: Add detailed source information for raw_text_snippet ---
                req['raw_snippet_origin_info'] = {
                    "chunk_index": i + 1,
                    "chunk_pages": page_numbers_in_chunk if page_numbers_in_chunk else []
                }

                if 'raw_text_snippet' not in req or not req['raw_text_snippet']:
                    req['raw_text_snippet'] = (
                        f"요구사항 관련 원본 텍스트 스니펫을 LLM으로부터 직접 추출하지 못했습니다. "
                        f"이 요구사항은 청크 #{i+1} (해당 청크에 포함된 원본 PDF 페이지 번호: {req['raw_snippet_origin_info']['chunk_pages']})에서 식별되었습니다. "
                        f"해당 청크의 시작 부분: \"{chunk_text[:250].strip()}...\""
                    )
                    req['raw_text_snippet_source_type'] = "auto_generated_fallback"
                else:
                    req['raw_text_snippet_source_type'] = "llm_extracted"
                # --- MODIFICATION END ---

                req['type'] = req.get('type', '기타')
                req['description'] = req.get('description', '설명 없음')
                req['acceptance_criteria'] = req.get('acceptance_criteria', '해당하는 경우 명시')
                req['priority'] = req.get('priority', '필수')
                req['responsible_module'] = req.get('responsible_module', '미정')
                final_semantic_chunks.append(req)

        except json.JSONDecodeError as e:
            print(f"LLM 응답 JSON 파싱 오류: {e}. 응답: {llm_response_content[:500]}...")
        except Exception as e:
            print(f"LLM API 호출 또는 처리 중 오류 발생: {e}")

    return final_semantic_chunks


# --- LLM으로 목차 파싱하는 함수 (이전 답변에서 제공 및 개선된 버전) ---
def parse_toc_with_llm(toc_raw_text, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    """
    LLM을 사용하여 목차 원문 텍스트를 파싱하여 구조화된 목차 항목을 추출합니다.
    """
    system_prompt = """
    당신은 PDF 문서에서 추출된 목차(Table of Contents)의 원시 텍스트를 분석하는 전문가입니다.
    주어진 텍스트를 파싱하여 각 목차 항목의 제목, 페이지 번호, 그리고 해당 항목이 '요구사항' 관련 내용을 담고 있을 가능성을 분석하여 JSON 형태로 반환해야 합니다.
    JSON의 최상위 레벨은 "toc_entries"라는 키를 가진 객체여야 하고, 그 키의 값은 목차 항목 객체들의 리스트여야 합니다.
    """
    user_prompt = f"""
    다음은 PDF에서 추출한 목차로 추정되는 텍스트입니다:

    --- 목차 원문 텍스트 시작 ---
    {toc_raw_text}
    --- 목차 원문 텍스트 끝 ---

    이 텍스트를 분석하여 JSON 객체를 반환해주세요. 이 객체는 "toc_entries"라는 키를 가져야 하며,
    이 키의 값은 각 목차 항목을 나타내는 객체들의 리스트여야 합니다.
    각 목차 항목 객체는 다음 키를 가져야 합니다:
    - "title": (문자열) 목차 항목의 전체 제목. 제목 앞의 번호(예: "1.", "II.", "가.")도 포함해주세요.
    - "page": (정수) 해당 항목의 시작 페이지 번호.
    - "is_requirement_related": (불리언) 제목이나 내용을 볼 때, 해당 항목이 '요구사항', '과업 범위', '제안 요청 상세', '기능 명세', '기술 요건' 등과 관련된 내용을 다룰 가능성이 높으면 true, 그렇지 않으면 false로 설정해주세요.

    예시 JSON 출력 형식:
    {{
      "toc_entries": [
        {{
          "title": "1. 사업 개요",
          "page": 5,
          "is_requirement_related": false
        }},
        {{
          "title": "III. 제안요청 내용",
          "page": 6,
          "is_requirement_related": true
        }},
        {{
          "title": "3. 상세 요구사항",
          "page": 11,
          "is_requirement_related": true
        }},
        {{
          "title": "* 보안 요구사항 별표",
          "page": 63,
          "is_requirement_related": true
        }}
      ]
    }}

    만약 주어진 텍스트가 유효한 목차로 보이지 않거나 항목을 전혀 파싱할 수 없다면,
    "toc_entries" 키의 값으로 빈 리스트 `[]`를 포함하는 JSON 객체를 반환해주세요. (예: {{"toc_entries": []}})
    """
    llm_response_content = "" # 오류 발생 시 로깅을 위해 미리 선언
    try:
        response = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        llm_response_content = response.choices[0].message.content
        # print(f"DEBUG: LLM RAW RESPONSE FOR TOC: {llm_response_content}")

        parsed_data = json.loads(llm_response_content)

        extracted_list = []
        if isinstance(parsed_data, dict) and "toc_entries" in parsed_data and isinstance(parsed_data["toc_entries"], list):
            extracted_list = parsed_data["toc_entries"]
        else:
            print(f"LLM 응답이 예상된 'toc_entries' 리스트를 포함하는 객체 형식이 아닙니다. 응답: {llm_response_content[:200]}")
            return [] # 빈 리스트 반환

        valid_entries = []
        for entry in extracted_list:
            if isinstance(entry, dict) and 'title' in entry and 'page' in entry:
                try:
                    entry['page'] = int(entry['page'])
                    entry['is_requirement_related'] = bool(entry.get('is_requirement_related', False)) # 명확히 불리언으로
                    valid_entries.append(entry)
                except ValueError:
                    print(f"경고: 페이지 번호 '{entry.get('page')}'를 정수로 변환할 수 없습니다. 항목 건너뜀: {entry.get('title')}")
                    continue
            else:
                print(f"경고: 필수 키(title, page)가 누락된 항목입니다. 건너뜀: {entry}")

        return valid_entries

    except json.JSONDecodeError as e:
        print(f"LLM 목차 파싱 응답 JSON 파싱 오류: {e}. 응답 미리보기: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"LLM API 호출 또는 처리 중 오류 발생 (목차 파싱): {e}")
        return []

# --- 새로운 헬퍼 함수들 ---
def extract_text_for_pages(full_text_with_pages, start_page_num, end_page_num):
    """
    전체 텍스트에서 특정 페이지 범위의 텍스트만 추출합니다.
    페이지 마커 '---PAGE_START_N---'를 사용합니다.
    """
    if end_page_num < start_page_num:
        return ""

    start_marker = f"---PAGE_START_{start_page_num}---"
    start_match = re.search(re.escape(start_marker), full_text_with_pages)

    if not start_match:
        return ""

    text_from_start_page = full_text_with_pages[start_match.start():]
    end_marker_exclusive = f"---PAGE_START_{end_page_num + 1}---"
    end_match = re.search(re.escape(end_marker_exclusive), text_from_start_page)

    if end_match:
        return text_from_start_page[:end_match.start()]
    else:
        return text_from_start_page


def get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages):
    """
    LLM으로 파싱된 목차에서 '상세 요구사항' 및 '보안 요구사항' 관련 섹션 정보를 추출합니다.
    """
    target_sections = []
    sorted_toc = sorted(parsed_toc_entries, key=lambda x: x.get('page', 0))
    keywords_to_find = {
        "상세 요구사항": {"min_pages": 5},
        "보안 요구사항 별표": {"min_pages": 1}
    }

    for i, entry in enumerate(sorted_toc):
        entry_title = entry.get('title', '').strip()
        entry_page = entry.get('page', 0)

        for keyword, props in keywords_to_find.items():
            if keyword in entry_title:
                start_page = entry_page
                end_page = total_pages
                next_major_section_page = total_pages + 1
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page :
                        next_major_section_page = next_entry_page
                        break
                end_page = min(total_pages, next_major_section_page -1)
                if end_page < start_page:
                    end_page = start_page
                
                target_sections.append({
                    'title': entry_title,
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 파싱 목차 기반: '{entry_title}' 섹션 (페이지 {start_page}-{end_page}) 식별")
                break 
    
    if not target_sections:
        print("키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...")
        for i, entry in enumerate(sorted_toc):
            if entry.get('is_requirement_related'):
                start_page = entry.get('page',0)
                end_page = total_pages
                next_major_section_page = total_pages + 1
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page:
                        if not next_entry.get('is_requirement_related', False): 
                            next_major_section_page = next_entry_page
                            break
                end_page = min(total_pages, next_major_section_page - 1)
                if end_page < start_page: end_page = start_page

                target_sections.append({
                    'title': entry.get('title', '요구사항 관련 섹션'),
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 'is_requirement_related' 플래그 기반: '{entry.get('title')}' 섹션 (페이지 {start_page}-{end_page}) 식별")

    if not target_sections:
        print("경고: LLM 파싱 목차에서 주요 요구사항 섹션을 찾지 못했습니다. 전체 문서를 대상으로 할 수 있습니다.")
        return [{'title': '전체 문서 (목차 분석 실패)', 'start_page': 1, 'end_page': total_pages}]

    return target_sections


def get_toc_raw_text_from_full_text(full_text_with_pages, toc_page_numbers=[2,3]):
    """
    전체 텍스트에서 지정된 목차 페이지들의 텍스트만 추출합니다.
    """
    toc_texts = []
    for page_num in toc_page_numbers:
        page_content = extract_text_for_pages(full_text_with_pages, page_num, page_num)
        if page_content:
            marker = f"---PAGE_START_{page_num}---\n"
            if page_content.startswith(marker):
                toc_texts.append(page_content[len(marker):])
            else: 
                toc_texts.append(page_content)
        else:
            print(f"경고: 목차 페이지로 지정된 {page_num} 페이지에서 텍스트를 찾을 수 없습니다.")

    if not toc_texts:
        return None
    return "\n".join(toc_texts)

# --- 메인 실행 블록 ---
if __name__ == "__main__":
    pdf_file_path = "./docs/제주은행 모바일뱅킹 재구축사업.pdf" # 실제 파일 경로
    if not os.path.exists(pdf_file_path):
        print(f"오류: PDF 파일을 찾을 수 없습니다 - {pdf_file_path}")
        exit()

    print("1. PDF 전체 텍스트 추출 중...")
    full_document_text, total_pages = extract_text_with_page_info(pdf_file_path)
    if not full_document_text or total_pages == 0:
        print(f"오류: PDF에서 텍스트를 추출하지 못했거나 페이지 수가 0입니다 ({pdf_file_path}).")
        exit()
    print(f"   PDF 전체 텍스트 추출 완료. 총 {total_pages} 페이지, 전체 텍스트 길이: {len(full_document_text)}자.")

    print("\n2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...")
    toc_raw_text = get_toc_raw_text_from_full_text(full_document_text, toc_page_numbers=[2, 3])

    target_sections_for_extraction = []
    if toc_raw_text:
        print(f"   목차 원문 텍스트 추출 완료 (길이: {len(toc_raw_text)}자).")
        print("\n3. LLM을 사용하여 목차(ToC) 파싱 중...")
        parsed_toc_entries = parse_toc_with_llm(toc_raw_text, client, llm_model=LLM_MODEL_FOR_PARSING)

        if parsed_toc_entries:
            print(f"   LLM 목차 파싱 완료. {len(parsed_toc_entries)}개의 목차 항목 식별.")
            print("\n4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...")
            target_sections_for_extraction = get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages)
        else:
            print("   LLM 목차 파싱 실패. 전체 문서를 대상으로 분석합니다.")
            target_sections_for_extraction = [{'title': '전체 문서 (LLM 목차 파싱 실패)', 'start_page': 1, 'end_page': total_pages}]
    else:
        print("   목차 원문 텍스트 추출 실패. 전체 문서를 대상으로 분석합니다.")
        target_sections_for_extraction = [{'title': '전체 문서 (목차 원문 추출 실패)', 'start_page': 1, 'end_page': total_pages}]

    if not target_sections_for_extraction: 
        print("오류: 분석할 대상 섹션을 정의하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    print("\n5. 식별된 주요 섹션으로부터 텍스트 결합 중...")
    all_requirements_related_text_parts = []
    for section_info in target_sections_for_extraction:
        print(f"   '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']}) 텍스트 추출 시도...")
        section_text = extract_text_for_pages(full_document_text, section_info['start_page'], section_info['end_page'])
        if section_text:
            all_requirements_related_text_parts.append(section_text)
            print(f"         '{section_info['title']}' 섹션 텍스트 추출 완료 (길이: {len(section_text)}자).")
        else:
            print(f"         경고: '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']})에서 텍스트를 추출하지 못했습니다.")

    if not all_requirements_related_text_parts:
        print("오류: 주요 요구사항 섹션에서 텍스트를 전혀 추출하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    combined_requirements_text = "\n\n".join(all_requirements_related_text_parts) 
    print(f"   주요 섹션 텍스트 결합 완료. 총 길이: {len(combined_requirements_text)}자.")

    print("\n6. 결합된 텍스트 초기 분할 중...")
    initial_chunks = initial_text_split(combined_requirements_text, chunk_size=3500, chunk_overlap=350)
    print(f"   총 {len(initial_chunks)}개의 초기 청크가 생성되었습니다.")

    if not initial_chunks:
        print("오류: 텍스트 분할 결과 청크가 없습니다. 프로그램을 종료합니다.")
        exit()

    print("\n7. LLM을 이용한 개발자 표준 요구사항 상세 추출 시작...")
    extracted_requirements = llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model=LLM_MODEL_FOR_PARSING)

    print("\n8. 후처리: ID 중복 제거 및 최종 정렬 (옵션)...")
    unique_requirements_by_id = {}
    processed_requirements = []
    if extracted_requirements:
        for req in extracted_requirements:
            req_id = req.get('id', str(uuid.uuid4())) 
            if req_id not in unique_requirements_by_id:
                unique_requirements_by_id[req_id] = req
            else:
                existing_req = unique_requirements_by_id[req_id]
                new_pages = req.get('source_pages', [])
                if new_pages:
                    existing_req_pages = existing_req.get('source_pages', [])
                    existing_req['source_pages'] = sorted(list(set(existing_req_pages + new_pages)))
                print(f"중복 ID '{req_id}' 감지. 정보 업데이트 시도.")
        processed_requirements = list(unique_requirements_by_id.values())
    else:
        print("   추출된 요구사항이 없습니다.")

    print(f"\n--- 최종 추출된 고유 요구사항 수: {len(processed_requirements)}개 ---")
    if processed_requirements:
        print("\n--- 추출된 요구사항 미리보기 (상위 3개) ---")
        for i, req_item in enumerate(processed_requirements[:3]): 
            print(json.dumps(req_item, ensure_ascii=False, indent=2))
            print("-" * 30)

        output_filename = "extracted_requirements_final.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(processed_requirements, f, ensure_ascii=False, indent=4)
        print(f"\n추출된 요구사항이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("최종적으로 추출된 요구사항이 없습니다.")

1. PDF 전체 텍스트 추출 중...
   PDF 전체 텍스트 추출 완료. 총 41 페이지, 전체 텍스트 길이: 30359자.

2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...
   목차 원문 텍스트 추출 완료 (길이: 2079자).

3. LLM을 사용하여 목차(ToC) 파싱 중...
   LLM 목차 파싱 완료. 17개의 목차 항목 식별.

4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...
키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...
LLM 'is_requirement_related' 플래그 기반: 'II 제안 요청 내역' 섹션 (페이지 4-17) 식별
LLM 'is_requirement_related' 플래그 기반: '1. 사업범위' 섹션 (페이지 4-17) 식별
LLM 'is_requirement_related' 플래그 기반: '2. 제안요건' 섹션 (페이지 7-17) 식별
LLM 'is_requirement_related' 플래그 기반: 'IV 제안서 작성 요령' 섹션 (페이지 23-25) 식별
LLM 'is_requirement_related' 플래그 기반: '1. 작성순서 및 세부지침' 섹션 (페이지 23-25) 식별
LLM 'is_requirement_related' 플래그 기반: '2. 세부 작성 방법' 섹션 (페이지 24-25) 식별

5. 식별된 주요 섹션으로부터 텍스트 결합 중...
   'II 제안 요청 내역' 섹션 (페이지 4-17) 텍스트 추출 시도...
         'II 제안 요청 내역' 섹션 텍스트 추출 완료 (길이: 12080자).
   '1. 사업범위' 섹션 (페이지 4-17) 텍스트 추출 시도...
         '1. 사업범위' 섹션 텍스트 추출 완료 (길이: 12080자).
   '2. 제안요건' 섹션 (페이지 7-17) 텍스트 추출 시도...
         '2. 제안요건' 섹션 텍스트 추출 완료 (길이:

In [ ]:
import fitz  # PyMuPDF
import re
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json
import uuid
from typing import List, Dict, Any, Tuple, Union

# --- OpenAI API 클라이언트 초기화 ---
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")
client = OpenAI(api_key=api_key)
LLM_MODEL = "gpt-4o"

# === 1. 전처리 함수 ===
def expert_extract_text_with_pages(pdf_path: str) -> Tuple[str, int]:
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text("text", sort=True) 
        full_text_with_pages.append(f"---PAGE_START_{page_num+1}---\n{text.strip()}")
    return "\n\n".join(full_text_with_pages), document.page_count

def expert_create_chunks(full_text: str, chunk_size: int = 4000, chunk_overlap: int = 500) -> List[Dict[str, Any]]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n---PAGE_START_\d+---\n", "\n\n\n", "\n\n", "\n", ". ", " ", ""],
        keep_separator=True
    )
    docs_langchain = text_splitter.create_documents([full_text])
    
    chunks = []
    for i, doc_lc in enumerate(docs_langchain):
        chunk_text = doc_lc.page_content
        page_numbers = sorted(list(set(int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text))))
        chunks.append({
            "chunk_id": i,
            "text": chunk_text,
            "source_pages_in_chunk": page_numbers
        })
    return chunks

# === 2. LLM 호출 헬퍼 함수 ===
def expert_llm_call(system_prompt: str, user_prompt: str, client_instance: OpenAI, model: str, expect_single_object: bool = False) -> Union[List[Dict[str, Any]], Dict[str, Any], None]:
    llm_response_content = ""
    try:
        response = client_instance.chat.completions.create(
            model=model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            max_tokens=4000 
        )
        llm_response_content = response.choices[0].message.content
        parsed_output = json.loads(llm_response_content)

        if expect_single_object:
            if isinstance(parsed_output, dict):
                return parsed_output
            else:
                if isinstance(parsed_output, list) and len(parsed_output) == 1 and isinstance(parsed_output[0], dict):
                    print(f"정보: 단일 JSON 객체를 예상했으나 리스트로 감싸인 객체를 받았습니다. 첫 번째 항목 사용. 내용: {str(parsed_output[0])[:100]}")
                    return parsed_output[0]
                print(f"경고: 단일 JSON 객체를 예상했으나 다른 타입/구조를 받았습니다: {type(parsed_output)}. 내용: {llm_response_content[:200]}")
                return None 
        else: 
            if isinstance(parsed_output, dict):
                for key, value in parsed_output.items():
                    if isinstance(value, list):
                        if all(isinstance(item, dict) for item in value):
                            return value
                print(f"경고: LLM이 JSON 객체를 반환했으나, 내부에 예상된 리스트를 찾지 못했습니다. 키: {list(parsed_output.keys())}, 내용: {str(parsed_output)[:200]}")
                return []
            elif isinstance(parsed_output, list):
                if all(isinstance(item, dict) for item in parsed_output):
                    return parsed_output
                else:
                    print(f"경고: LLM이 리스트를 반환했으나, 일부 항목이 딕셔너리가 아닙니다.")
                    return [item for item in parsed_output if isinstance(item, dict)]
            else:
                print(f"경고: LLM으로부터 예상치 못한 JSON 루트 형식 응답 (리스트 또는 객체가 아님): {llm_response_content[:200]}...")
                return []

    except json.JSONDecodeError as e:
        print(f"LLM 응답 JSON 파싱 오류: {e}. 응답: {llm_response_content[:500]}...")
        return None if expect_single_object else []
    except Exception as e:
        print(f"LLM API 호출 또는 처리 중 오류 발생: {type(e).__name__} {e}")
        if llm_response_content:
            print(f"오류 발생 시 LLM 응답 일부: {llm_response_content[:500]}...")
        return None if expect_single_object else []

# === 3. 패스별 시스템 프롬프트 정의 ===
def get_expert_system_prompt(pass_number=1):
    system_prompt_pass1 = """
    당신은 RFP(제안요청서) 문서에서 시스템 요구사항을 전문적으로 분석하고 개발 표준에 맞춰 구조화하는 시니어 비즈니스 분석가입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 일부이며, 여기서 모든 기능적, 비기능적, 성능, 보안, 데이터, 제약사항 등 **모든 종류의 시스템 요구사항 후보**를 추출해야 합니다.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.
    -   **id**: 고유 식별자. (RFP 원문에 ID가 있다면 그것을 우선 사용)
    -   **type**: 요구사항의 유형. ("기능적", "비기능적", "성능", "보안", "데이터", "제약사항", "시스템 장비 구성", "컨설팅", "테스트", "품질", "프로젝트 관리", "프로젝트 지원", "운영", "표준준수", "기타")
    -   **description**: 요구사항에 대한 명확하고 간결한 설명. 원본 텍스트의 핵심 내용을 정확히 반영해야 합니다.
    -   **acceptance_criteria**: 이 요구사항이 충족되었음을 검증할 수 있는 구체적인 테스트 조건이나 결과. (추론이 어렵거나 해당 없으면 유연하게 작성)
    -   **priority**: 요구사항의 중요도. (텍스트에 명시되어 있다면 그대로 사용, 아니면 일반적으로 "필수"로 추정)
    -   **responsible_module**: 이 요구사항이 주로 영향을 미치거나 구현될 것으로 예상되는 시스템/애플리케이션의 주요 모듈 또는 영역. (추론이 어렵거나 해당 없으면 유연하게 작성)
    -   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트.
    -   **raw_text_snippet**: 이 요구사항이 포함된 원본 텍스트 스니펫 (추출 근거가 된 문장 또는 단락 전체).

    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 요구사항이 없다면 빈 배열 `[]`을 반환하십시오. (실제로는 response_format 지침에 따라 {"키": [...]} 형태로 반환될 수 있으며, 이 경우 해당 "키"의 값을 사용합니다.)

    **패스 1 중요 지침 (매우 관대하고 포괄적으로 추출):**
    1.  텍스트 전체를 면밀히 검토하여, 명시적인 요구사항뿐만 아니라 **묵시적으로 요구사항으로 해석될 수 있는 모든 내용**을 포함해 주십시오.
    2.  요구사항은 특정 섹션에만 국한되지 않을 수 있습니다. 문서의 **어떤 부분에서든** 발견될 수 있는 요구사항을 모두 찾아내야 합니다.
    3.  **단 하나의 잠재적인 요구사항도 놓치지 않는 것이 매우 중요합니다.** 만약 어떤 내용이 요구사항인지 아닌지 판단하기 애매하거나 경계선에 있는 경우에도, **요구사항일 가능성이 조금이라도 있다면 일단 추출**하고, 'type'은 가장 적절하다고 생각되는 유형 또는 '기타'로 지정해주십시오.
    4.  **목표는 최대한 많은 후보를 찾는 것입니다. 정밀도보다는 재현율(Recall)을 극대화해주세요.**

    5.  **다양한 유형의 요구사항 식별 (매우 중요):** 시스템의 직접적인 기능/성능 외에도, 다음과 같은 내용들을 모두 중요한 요구사항으로 간주하고 추출하십시오:
        a.  **제안 제품(H/W, S/W) 조건:** 제품의 상태(예: 단종 여부), 변경 가능성, 공급업체 안정성, 기술 지원 조건, 라이선스 정책, 소스코드 제공 의무 등.
            (예: "생산중단 계획이 없는 제품을 제안하여야 함", "제조사의 공급 확약서 및 기술지원 확약서 첨부해야 함", "프로그램 소스를 당행에 제공하여야 함")
        b.  **시스템 운영 조건:** 관련 규정 준수 의무, 시스템 관리/장애처리/모니터링/백업 등의 운영 방안 범위, 역할/책임 명시 의무, 솔루션 연계 방안 제시 의무 등.
            (예: "대내외 전산 관리 규정을 준수하여야 함", "운영조직은 역할 및 담당 책임자를 명확히 명시해야 함")
        c.  **표준화 및 정책 준수:** 은행 내부 특정 표준(통합백업, SMS, NMS, 통합콘솔, 형상관리, 채널통합(MCI) 및 EAI표준, 데이터 표준화 지침 및 메타시스템 정책, 웹 표준 등) 준수 의무, 특정 개발 방법론 적용 의무 등.
            (예: "당행의 통합백업정책을 준수해야 함", "웹 표준기술(HTML5)을 사용해야 함")
        d.  **프로젝트 관리 및 수행 절차:** 단계별 목표/절차/산출물 제시 의무, 테스트 방안/계획 제시 및 승인 의무, 보고 의무, 인력 관리 조건, 보안 대책 수립 및 준수 의무 등.
            (예: "일정별 개발 방안을 제시해야 함", "테스트 계획을 구체적으로 수립하고 당행의 승인을 받아야 함")
        e.  **기타 계약 조건 및 제약사항:** 지식재산권, 법적 책임, 하자보수 조건, 추가 요소 지원 의무, 도입 누락 책임 등.
            (예: "도입 누락에 따른 책임은 제안사에 있음", "법률상의 문제에 대한 일체의 책임을 져야 함")

    6.  **JSON 필드 작성 유연성:** 위 5번 항목과 같은 유형의 요구사항에 대해 `type`은 "제약사항", "품질", "프로젝트 관리", "프로젝트 지원", "보안", "운영", "표준준수", "기타" 등으로 분류하고, `acceptance_criteria`는 "관련 문서 제출 및 승인", "해당 표준/정책 준수여부 확인", "관련 확약서 제출", "계약서 명시 및 이행" 등으로 기술하거나, 명확한 기준 설정이 어려울 경우 "해당 없음" 또는 "세부 협의" 등으로 작성해도 됩니다. `responsible_module`도 "제안사 전체", "프로젝트 팀" 등으로 포괄적으로 지정하거나 비워둘 수 있습니다. **가장 중요한 것은 `description`에 원본 요구사항의 핵심 내용을 정확히 담는 것입니다.** `priority`는 명시되어 있지 않으면 "필수"로 간주하십시오.
    
    LLM 응답은 `{"requirements_found": [...]}` 형태의 JSON 객체여야 하며, `requirements_found` 키의 값이 실제 요구사항 객체들의 배열입니다.
    """

    system_prompt_pass2 = """
    당신은 RFP(제안요청서) 문서를 매우 꼼꼼하게 재검토하여, 이전 분석에서 놓쳤을 수 있는 추가적인 시스템 요구사항을 발굴하는 전문가입니다.
    사용자가 제공하는 텍스트(이전 패스에서 이미 한 번 검토되었을 수 있음)를 새로운 관점에서 다시 한번 분석하여, 숨겨져 있거나 간과된 요구사항을 찾아내 주십시오.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.
    (JSON 구조 설명은 Pass 1과 동일)
    -   **id**: ...
    -   **type**: ...
    -   **description**: ...
    -   **acceptance_criteria**: ...
    -   **priority**: ...
    -   **responsible_module**: ...
    -   **source_pages**: ...
    -   **raw_text_snippet**: ...
    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 추가 요구사항이 없다면 빈 배열 `[]`을 반환하십시오. (실제로는 response_format 지침에 따라 {"키": [...]} 형태로 반환될 수 있으며, 이 경우 해당 "키"의 값을 사용합니다.)
    "JSON 객체의 id를 제외한 모든 값은 한국어로 기술되어야 합니다."
    **패스 2 중요 지침 (새로운 관점에서 누락된 부분 찾기):**
    1.  이 텍스트는 이미 요구사항 추출 시도가 있었을 수 있다는 점을 감안하고, **이전에 명백하게 추출되지 않았을 것 같은 요구사항들**에 집중해주십시오.
    2.  특히, 문맥 속에 숨겨져 있거나, 비기능적 특성(성능, 보안, 사용성, 안정성 등), 제약 조건, 품질 요구사항, 데이터 관련 요구사항 등 **일반적인 기능 목록에서 쉽게 누락될 수 있는 유형**의 요구사항에 주의를 기울여 주십시오. Pass 1에서 놓쳤을 만한 세부적인 정책 준수, 운영 절차, 제품 조건 등을 다시 한번 확인해주십시오.
    3.  애매하거나 해석의 여지가 있는 부분도, 요구사항으로 볼 수 있는 합리적인 근거가 있다면 포함시켜 주십시오.
    4.  **첫 번째 분석에서 놓쳤을 만한 것들을 찾아내는 것이 목표입니다.** `description`은 명확하게, `acceptance_criteria`는 가능한 구체적으로 작성하되, 어려우면 "세부 협의" 등으로 명시하십시오.
    
    LLM 응답은 `{"additional_requirements_found": [...]}` 형태의 JSON 객체여야 하며, `additional_requirements_found` 키의 값이 실제 요구사항 객체들의 배열입니다.
    """
    if pass_number == 1:
        return system_prompt_pass1
    elif pass_number == 2:
        return system_prompt_pass2
    else: 
        return system_prompt_pass1


# === 4. 패스 1 실행 함수 ===
def expert_pass1_identify_candidates(chunks: List[Dict[str, Any]], client_instance: OpenAI, model: str) -> List[Dict[str, Any]]:
    system_prompt = get_expert_system_prompt(pass_number=1)
    all_candidates = []
    for i, chunk_data in enumerate(chunks):
        chunk_text = chunk_data["text"]
        chunk_pages = chunk_data["source_pages_in_chunk"]
        print(f"--- 전문가 패스 1: 후보 식별 중 - 청크 {i+1}/{len(chunks)} (원본 페이지: {chunk_pages}) ---")
        
        user_prompt = f"""
        다음은 RFP 문서의 일부 텍스트 청크입니다. 이 청크 내에서 시스템 프롬프트의 지침에 따라 모든 '잠재적 요구사항 후보' 텍스트를 식별하여 요청된 JSON 형식으로 응답해주십시오. 
        `source_pages` 필드에는 이 청크에 해당하는 원본 페이지 번호인 {chunk_pages}를 사용하십시오. (후보 텍스트가 특정 페이지에서 시작된 것을 알 수 있다면 해당 페이지 번호만 사용해도 좋습니다.)

        --- 텍스트 청크 ---
        {chunk_text}
        """
        
        # Pass 1은 LLM이 {"requirements_found": [...]} 로 반환하도록 유도
        # expert_llm_call은 이 내부 리스트를 반환함
        candidates_from_chunk_list = expert_llm_call(system_prompt, user_prompt, client_instance, model, expect_single_object=False) 
        
        if isinstance(candidates_from_chunk_list, list):
            for cand_idx, cand in enumerate(candidates_from_chunk_list):
                if isinstance(cand, dict) and "description" in cand and "type" in cand: # Pass1도 기본 구조는 채우도록 유도
                    cand["pass_info"] = {"pass": 1, "chunk_id": chunk_data["chunk_id"], "candidate_index_in_chunk": cand_idx}
                    if not cand.get("source_pages") or not isinstance(cand.get("source_pages"), list) :
                        cand["source_pages"] = chunk_pages # 페이지 정보가 없다면 청크 전체 페이지로
                    all_candidates.append(cand)
                else:
                    print(f"경고 (패스1, 청크 {i+1}): 후보 항목에 주요 키 또는 올바른 타입 누락. 항목: {str(cand)[:100]}")
        else:
            print(f"경고 (패스1, 청크 {i+1}): expert_llm_call로부터 리스트가 아닌 결과 반환. 결과: {str(candidates_from_chunk_list)[:100]}")

    print(f"--- 전문가 패스 1 완료: 총 {len(all_candidates)}개의 잠재적 요구사항 후보 식별 ---")
    return all_candidates

# === 5. 패스 2 실행 함수 ===
def expert_pass2_refine_and_add(
    chunks: List[Dict[str, Any]], # 패스 1과 동일한 청크 사용
    client_instance: OpenAI, 
    model: str
) -> List[Dict[str, Any]]:
    system_prompt = get_expert_system_prompt(pass_number=2)
    additional_requirements = []
    for i, chunk_data in enumerate(chunks): # 전체 청크를 다시 한번 검토
        chunk_text = chunk_data["text"]
        chunk_pages = chunk_data["source_pages_in_chunk"]
        print(f"--- 전문가 패스 2: 추가/보완 추출 중 - 청크 {i+1}/{len(chunks)} (원본 페이지: {chunk_pages}) ---")

        user_prompt = f"""
        다음은 RFP 문서의 일부 텍스트 청크입니다. 이전에 이미 한 번 검토되었을 수 있습니다.
        시스템 프롬프트의 지침에 따라, 이 청크에서 **이전에 놓쳤을 가능성이 있는 추가적인 요구사항**을 찾아내어 요청된 JSON 형식으로 응답해주십시오.
        `source_pages` 필드에는 이 청크에 해당하는 원본 페이지 번호인 {chunk_pages}를 사용하십시오.

        --- 텍스트 청크 ---
        {chunk_text}
        """
        
        # Pass 2도 LLM이 {"additional_requirements_found": [...]} 로 반환하도록 유도
        newly_found_reqs_list = expert_llm_call(system_prompt, user_prompt, client_instance, model, expect_single_object=False)

        if isinstance(newly_found_reqs_list, list):
            for req_idx, req in enumerate(newly_found_reqs_list):
                if isinstance(req, dict) and "description" in req and "type" in req:
                    req["pass_info"] = {"pass": 2, "chunk_id": chunk_data["chunk_id"], "item_index_in_chunk": req_idx}
                    if not req.get("source_pages") or not isinstance(req.get("source_pages"), list) :
                        req["source_pages"] = chunk_pages
                    additional_requirements.append(req)
                else:
                    print(f"경고 (패스2, 청크 {i+1}): 항목에 주요 키 또는 올바른 타입 누락. 항목: {str(req)[:100]}")
        else:
             print(f"경고 (패스2, 청크 {i+1}): expert_llm_call로부터 리스트가 아닌 결과 반환. 결과: {str(newly_found_reqs_list)[:100]}")


    print(f"--- 전문가 패스 2 완료: {len(additional_requirements)}개의 추가/보완 요구사항 식별 ---")
    return additional_requirements


# === 6. 후처리 함수 (수정됨) ===
def expert_post_process_requirements(requirements: List[Dict[str, Any]], exclude_types: List[str] = None) -> List[Dict[str, Any]]:
    if exclude_types is None:
        exclude_types = [] 
    
    print(f"\n--- 전문가 후처리 시작: 초기 {len(requirements)}개 항목 ---")
    if not requirements:
        return []

    # 0. 지정된 타입 필터링
    if exclude_types:
        initial_count_before_type_filter = len(requirements)
        requirements = [req for req in requirements if req.get("type") not in exclude_types]
        print(f"    타입 기반 필터링: '{', '.join(exclude_types)}' 타입 제외 후 {len(requirements)}개 항목 (이전: {initial_count_before_type_filter}개)")

    # 1. 내용 기반 중복 제거
    unique_reqs_by_content = {}
    for req in requirements:
        desc_key = "".join(req.get('description', '').strip().lower().split())[:80]
        snippet_key = "".join(req.get('raw_text_snippet', '').strip().lower().split())[:80]
        page_key_list = sorted(list(set(int(p) for p in req.get('source_pages', []) if isinstance(p, (int,str)) and str(p).isdigit()))) # 정수 변환 추가
        page_key = "_p" + str(page_key_list[0]) if page_key_list else "_p0"
        content_key = f"{desc_key}_{snippet_key}_{page_key}" # 더 정확한 중복 판단을 위해 키 조합

        original_req_id = req.get('id', '') # LLM이 부여한 ID 또는 패스에서 생성된 임시ID

        if content_key not in unique_reqs_by_content:
            req["temp_merged_ids"] = [original_req_id] # 병합될 ID들 추적
            unique_reqs_by_content[content_key] = req
        else:
            existing_req = unique_reqs_by_content[content_key]
            print(f"    내용 기반 중복 발견. 키: '{content_key[:50]}...'. 기존ID: '{existing_req.get('id', 'N/A')}', 새ID: '{original_req_id}'. 정보 병합.")
            
            new_pages = [int(p) for p in req.get('source_pages', []) if isinstance(p, (int,str)) and str(p).isdigit()]
            existing_pages = [int(p) for p in existing_req.get('source_pages', []) if isinstance(p, (int,str)) and str(p).isdigit()]
            existing_req['source_pages'] = sorted(list(set(existing_pages + new_pages)))

            if len(req.get('description', '')) > len(existing_req.get('description', '')):
                existing_req['description'] = req.get('description')
            if len(req.get('raw_text_snippet', '')) > len(existing_req.get('raw_text_snippet', '')):
                 existing_req['raw_text_snippet'] = req.get('raw_text_snippet')
            
            # 패스 정보 활용하여 병합 (예: pass_info가 더 나중 것으로 업데이트)
            current_pass_info = req.get('pass_info', {})
            existing_pass_info = existing_req.get('pass_info', {})
            if current_pass_info.get('pass', 0) > existing_pass_info.get('pass', 0):
                existing_req['pass_info'] = current_pass_info
                # 더 나중 패스의 type, acceptance_criteria 등을 우선 적용할 수 있음
                if req.get('type') != '기타' or existing_req.get('type') == '기타': existing_req['type'] = req.get('type')
                ac_placeholder = ['추후 분석 필요', '해당하는 경우 명시', '해당 없음', '세부 협의']
                if req.get('acceptance_criteria') not in ac_placeholder or existing_req.get('acceptance_criteria') in ac_placeholder:
                    existing_req['acceptance_criteria'] = req.get('acceptance_criteria')

            if 'temp_merged_ids' in existing_req and isinstance(existing_req['temp_merged_ids'], list):
                 existing_req['temp_merged_ids'].append(original_req_id)
            else:
                 existing_req['temp_merged_ids'] = [existing_req.get('id',''), original_req_id]


    deduplicated_requirements = list(unique_reqs_by_content.values())
    print(f"    내용 기반 중복 제거 후: {len(deduplicated_requirements)}개 항목")

    # 2. 최종 ID 부여 및 정제
    final_requirements = []
    id_counters = {} 
    type_prefixes = {
        "기능적": "FUNC", "비기능적": "NFR", "성능": "PERF", "보안": "SEC",
        "데이터": "DATA", "제약사항": "CONS", "인터페이스": "IF", "운영": "OPS",
        "테스트": "TEST", "품질": "QUAL", "프로젝트 관리": "PM", 
        # "프로젝트 지원": "PSUP", # 제외 요청됨
        # "정책준수": "POL",   # 제외 요청됨
        # "기타": "ETC"        # 제외 요청됨
    }
    # 제외되지 않는 타입에 대해서만 ID 접두사 유지
    valid_type_prefixes = {k: v for k, v in type_prefixes.items() if k not in exclude_types}


    for i, req_dict in enumerate(deduplicated_requirements):
        req_type = req_dict.get("type", "기타")
        
        # 만약 여기서도 exclude_types에 걸리는 타입이 있다면 최종 제외 (안전장치)
        if req_type in exclude_types:
            print(f"    최종 필터링: '{req_type}' 타입 항목 ID '{req_dict.get('id')}' 제외.")
            continue

        prefix_candidate = req_type[:3].upper().replace(" ", "").replace("_","") if req_type != "기타" else "ETC"
        prefix = valid_type_prefixes.get(req_type, prefix_candidate if prefix_candidate else "UNK") # Unknown prefix
        
        if prefix not in id_counters:
            id_counters[prefix] = 0
        id_counters[prefix] += 1
        
        req_dict["id"] = f"{prefix}-{id_counters[prefix]:03d}"
        
        req_dict.pop("pass_info", None) # 임시 필드 제거
        req_dict.pop("temp_merged_ids", None)
        
        final_requirements.append(req_dict)

    final_requirements.sort(key=lambda x: (min(x['source_pages']) if x.get('source_pages') else float('inf'), x.get('id', '')))
    
    print(f"--- 전문가 후처리 완료: 최종 {len(final_requirements)}개 요구사항 ---")
    return final_requirements


# === 메인 실행 로직 ===
if __name__ == "__main__":
    pdf_file_path = "./docs/제주은행 모바일뱅킹 재구축사업.pdf" 
    output_filename_template = "extracted_requirements_expert_v4_filtered_{types}.json"

    print(f"--- RFP 요구사항 추출 전문가 시스템 시작 (입력 파일: {pdf_file_path}) ---")

    full_text, total_pages = expert_extract_text_with_pages(pdf_file_path)
    if not full_text or total_pages == 0:
        print(f"오류: PDF에서 텍스트를 추출하지 못했거나 페이지 수가 0입니다.")
        exit()
    print(f"1. PDF 텍스트 추출 완료: 총 {total_pages} 페이지, 전체 텍스트 길이 {len(full_text)}자.")

    chunks = expert_create_chunks(full_text, chunk_size=3800, chunk_overlap=800) 
    if not chunks:
        print("오류: 문서에서 청크를 생성하지 못했습니다.")
        exit()
    print(f"2. 문서 청킹 완료: 총 {len(chunks)}개 청크 생성 (overlap: 800).")

    chunks_map = {chunk["chunk_id"]: chunk for chunk in chunks}

    print("\n--- 패스 1: 광범위하고 포괄적인 요구사항 후보 식별 시작 ---")
    pass1_requirements = expert_pass1_identify_candidates(chunks, client, LLM_MODEL)
    if not pass1_requirements:
        print("패스 1에서 잠재적 요구사항 후보를 찾지 못했습니다.")
    
    print("\n--- 패스 2: 추가 탐색 및 보완 추출 시작 ---")
    # 패스2는 패스1과 동일한 청크를 다시 보며 다른 관점에서 찾음
    pass2_requirements = expert_pass2_refine_and_add(chunks, client, LLM_MODEL)
    if not pass2_requirements:
        print("패스 2에서 추가/보완 요구사항을 찾지 못했습니다.")

    all_combined_requirements = pass1_requirements + pass2_requirements
    print(f"\n모든 패스에서 총 {len(all_combined_requirements)}개의 요구사항 후보 수집 (중복 포함).")
    
    # <<< --- 사용자 요청 반영: 특정 타입 제외 --- >>>
    types_to_exclude_user_defined = ["기타", "산출물", "정책준수", "프로젝트 지원"] 
    # <<< ------------------------------------ >>>

    print(f"\n--- 후처리 작업 시작 (제외할 타입: {types_to_exclude_user_defined}) ---")
    final_processed_requirements = expert_post_process_requirements(all_combined_requirements, exclude_types=types_to_exclude_user_defined)

    if final_processed_requirements:
        print(f"\n--- 최종 추출된 고유 요구사항: {len(final_processed_requirements)}개 ---")
        print("\n--- 상위 5개 요구사항 미리보기 ---")
        for i, req_item in enumerate(final_processed_requirements[:5]):
            print(json.dumps(req_item, ensure_ascii=False, indent=2))
            if i < 4: print("-" * 20)
        
        excluded_types_str = "_".join(sorted(types_to_exclude_user_defined)).replace(" ","") if types_to_exclude_user_defined else "none"
        output_filename = output_filename_template.format(types=excluded_types_str)
        
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(final_processed_requirements, f, ensure_ascii=False, indent=4)
            print(f"\n최종 추출된 요구사항이 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
        except IOError as e:
            print(f"오류: '{output_filename}' 파일 저장 중 문제가 발생했습니다: {e}")
    else:
        print("\n최종적으로 추출된 요구사항이 없습니다.")

    print("--- RFP 요구사항 추출 전문가 시스템 종료 ---")

--- RFP 요구사항 추출 전문가 시스템 시작 (입력 파일: ./docs/제주은행 모바일뱅킹 재구축사업.pdf) ---
1. PDF 텍스트 추출 완료: 총 54 페이지, 전체 텍스트 길이 57553자.
2. 문서 청킹 완료: 총 19개 청크 생성 (overlap: 800).

--- 패스 1: 광범위하고 포괄적인 요구사항 후보 식별 시작 ---
--- 전문가 패스 1: 후보 식별 중 - 청크 1/19 (원본 페이지: [1, 2]) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 2/19 (원본 페이지: [3]) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 3/19 (원본 페이지: []) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 4/19 (원본 페이지: [4]) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 5/19 (원본 페이지: []) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 6/19 (원본 페이지: [5, 6, 7]) ---
--- 전문가 패스 1: 후보 식별 중 - 청크 7/19 (원본 페이지: [8, 9]) ---
LLM 응답 JSON 파싱 오류: Unterminated string starting at: line 331 column 33 (char 12338). 응답: {
    "requirements_found": [
        {
            "id": "REQ-001",
            "type": "기능적",
            "description": "앱 구동 및 로그인, 조회, 이체에 대한 속도 개선",
            "acceptance_criteria": "앱 구동 및 로그인, 조회, 이체 시 속도 개선이 확인됨",
            "priority": "필수",
            "responsible_module": "모바일 앱",
            "source_pages": [8],
            "raw_text_sni

ValueError: min() arg is an empty sequence